#**Image Resolution Upscaler based on Generative Adversarial Networks (SRGAN)**

Hello,

My name is **Anas HAMOUTNI** and I am a **Moroccan Statistician**.

This notebook is an implementation of Single Image Super-Resolution based on the **SRGAN paper**: https://arxiv.org/pdf/1609.04802.pdf, we use the **ESRGAN model** from the repo: https://github.com/xinntao/ESRGAN/tree/master/models

Super-Resolution aims to increase an image resolution using the generative power of **GAN**s (**[Generative Adversarial Networks](https://en.wikipedia.org/wiki/Generative_adversarial_network)**).






In [1]:
#@title ##**Install all the necessary libraries** { display-mode: "form" }

!pip install torchvision==0.14
!pip install torch==1.13
from IPython.display import clear_output
from google.colab import files
import imageio
import cv2
import os
import torch
import fastai
import os.path as osp
from PIL import Image
from tqdm import tqdm
from os import path
import numpy as np
import sys
import glob
from google.colab import files
from IPython import display as ipythondisplay
from IPython.display import Image as ipythonimage
torch.backends.cudnn.benchmark=True
%matplotlib inline
clear_output()

In [2]:
#@title ##**Clone the repository and download the necessary components** { display-mode: "form" }
%cd /content
!git clone https://github.com/xinntao/ESRGAN.git
%cd /content/ESRGAN
!git checkout tags/old-arch
model_url = "https://www.dropbox.com/s/vouc15j8jjp2o5n/RRDB_ESRGAN_x4_old_arch.pth?dl=0"
!wget $model_url --content-disposition -P models
import architecture as arch
import os.path
!mkdir frames #make frames directory
clear_output()

In [3]:
#@title ##**Upload an Image** { display-mode: "form" }
%cd /content/ESRGAN/frames/ 
uploaded = files.upload() #upload image in frames folder
imgName = next(iter(uploaded)) #get image name



/content/ESRGAN/frames


Saving cropped-WhatsApp-Image-2020-09-17-at-18.48.07.jpeg to cropped-WhatsApp-Image-2020-09-17-at-18.48.07.jpeg


In [4]:
#@title ##**Upscale resolution** { display-mode: "form" }
#@markdown **How many times to upscale the resolution:**
upscale_factor = 4 #@param {type: "slider", min: 4, max: 8}

%cd /content/ESRGAN
%env CUDA_VISIBLE_DEVICES=0
device = torch.device('cuda') #torch.device enables you to specify the device type responsible to load a tensor into memory
model = arch.RRDB_Net(3, 3, 64, 23, gc=32, upscale=upscale_factor, norm_type=None, act_type='leakyrelu', \
                        mode='CNA', res_scale=1, upsample_mode='upconv') #Initializing the ESRGAN model
model.load_state_dict(torch.load('models/{:s}'.format('RRDB_ESRGAN_x4_old_arch.pth')), strict=True) #Loads a model's parameter dictionary using a deserialized state_dict
model.eval() #is a kind of switch for some specific layers/parts of the model that behave differently during training and inference (evaluating) time. For example, Dropouts Layers, BatchNorm Layers etc. You need to turn off them during model evaluation, and .eval() will do it for you.
for k, v in model.named_parameters():
    v.requires_grad = False #requires_grad_() 's main use case is to tell autograd to begin recording operations on a Tensor tensor . If tensor has requires_grad=False (because it was obtained through a DataLoader, or required preprocessing or initialization), tensor.
model = model.to(device) #convert the initialized model to a CUDA optimized model

base = os.path.splitext(os.path.basename("frames/"+imgName))[0]
path="frames/"+imgName #path to uploaded image
img = cv2.imread(path, cv2.IMREAD_COLOR) #cv2.imread() method loads an image from the specified file.  It returns a numpy.ndarray after loading the image successfully. 
img = img * 1.0 / 255 #scale image
img = torch.from_numpy(np.transpose(img[:, :, [2, 1, 0]], (2, 0, 1))).float() #Creates a Tensor from a numpy
img_LR = img.unsqueeze(0) #unsqueeze turns an n.d. tensor into an (n+1).d. one by adding an extra dimension of depth 1 (axis=0).
img_LR = img_LR.to(device) #convert the image to a CUDA optimized image

output = model(img_LR).data.squeeze().float().cpu().clamp_(0, 1).numpy() #submit the optimized image to the model
output = np.transpose(output[[2, 1, 0], :, :], (1, 2, 0))
output = (output * 255.0).round() #unscale image 
path = 'results/{:s}_rlt.png'.format(base) #output path
cv2.imwrite(path, output) #save output
clear_output()

from google.colab import files
files.download(path) #download output from its path


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>